# Feature Selection

In this short (compared to Feature Engineering one) notebook, I'll continue data preparation process. Here I'll try to "filter down" bloated dataset that we got at the end of previous notebook. Various techniques will be applied to select most useful for prediction features. Feature Selection should not be neglected, as one can benefit from it a lot: it decreases overfitting, reduces training time, makes models simpler, lifts the curse of dimensionality (partialy) and may even improve accuracy. There are dozens of FS methods, grouped by types (filter, wrapper, embedded), but I'll use simple and effective ones. The reason: I don't want to loose very much data and don't want wait for hours evaluating feature subsets. Check [this page](https://h2o.ai/wiki/feature-selection/) for more info on FS.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
sns.set()
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
import gc
from imports import *

In [3]:
train=pd.read_csv('../data/full_train.csv', engine='pyarrow')
train=convert_types(train, print_info=True)
train.shape

Original Memory Usage: 3.37 gb.
New Memory Usage: 1.71 gb.


(307511, 1369)

Note: with huge files like this, it may be a good idea to [use some formats other than csv](https://pythonspeed.com/articles/pandas-read-csv-fast/). It'll save you a lot if time. 

## Correlation

One of the simplest methods of feature selection is to remove highly correlated features. Presence of this features can negatively affect model's ability to learn, generalize. Hence, we should remove them. One should select the treshold themselves, but it's usually 0.8-0.9.

In [4]:
corr_mat=train.corr().abs()
upper=corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(np.bool))
upper.head()

<ipython-input-4-e9ccfe60ee12>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper=corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(np.bool))


,,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NEW_AMT_CREDIT_TO_AMT_INCOME,NEW_AMT_CREDIT_TO_AMT_ANNUITY,NEW_AMT_CREDIT_TO_AMT_GOODS_PRICE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_CONTRACT_TYPE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_HOUSING_TYPE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_ORGANIZATION_TYPE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_EDUCATION_TYPE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_GENDER,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_FAMILY_STATUS,NEW_AMT_CREDIT_TO_MEAN_AMT_INCOME_BY_AGE_GROUP,NEW_AMT_INCOME_BY_AGE_GROUP,NEW_AMT_INCOME_BY_CNT_CHILD,NEW_AMT_INCOME_BY_CNT_FAM_MEMBERS,NEW_AMT_INCOME_BY_AGE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_CONTRACT_TYPE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_HOUSING_TYPE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_ORGANIZATION_TYPE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_EDUCATION_TYPE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_GENDER,NEW_AMT_INCOME_TO_MEAN_AMT_INCOME_BY_AGE_GROUP,NEW_DOC_FLAG_MEAN,NEW_DOC_FLAG_SUM,NEW_CONTACT_FLAG_MEAN,NEW_CONTACT_FLAG_SUM,NEW_ADDRESS_FLAG_MEAN,NEW_ADDRESS_FLAG_SUM,NEW_OWN_CAR_REALTY_COMBINATION,NEW_AGE_TO_MEAN_AGE_BY_FLAG_OWN_REALTY,NEW_AGE_TO_MEAN_AGE_BY_FLAG_OWN_CAR,NEW_AGE_TO_MEAN_AGE_BY_HOUSING_TYPE,NEW_DAYS_EMPLOYED_TO_DAYS_BIRTH,NEW_DAYS_REGISTRATION_TO_DAYS_BIRTH,NEW_OWN_CAR_AGE_TO_DAYS_BIRTH,NEW_OWN_CAR_AGE_TO_DAYS_EMPLOYED,NEW_DAYS_LAST_PHONE_CHANGE_TO_DAYS_BIRTH,NEW_DAYS_LAST_PHONE_CHANGE_TO_DAYS_EMPLOYED,NEW_CNT_CHILD_TO_CNT_FAM_MEMBERS,NEW_EXT_SOURCES_MEAN,NEW_EXT_SOURCES_STD,NEW_DAYS_CHANGE_MEAN,NEW_REGION_RATING_CLIENT_MEAN,NEW_30_CNT_SOCIAL_CIRCLE_MEAN,NEW_60_CNT_SOCIAL_CIRCLE_MEAN,bureau_DAYS_CREDIT_sum,bureau_DAYS_ENDDATE_FACT_sum,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_ENDDATE_FACT_min,bureau_DAYS_ENDDATE_FACT_mean,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_UPDATE_sum,bureau_DAYS_ENDDATE_FACT_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_UPDATE_max,bureau_CNT_CREDIT_PROLONG_min,bureau_CREDIT_DAY_OVERDUE_min,bureau_AMT_CREDIT_SUM_OVERDUE_min,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_sum,bureau_AMT_CREDI

In [6]:
upper.to_csv('../data/corr_mat.csv')

In [12]:
upper=pd.read_csv('../data/corr_mat.csv').iloc[:, 1:]
upper.head()

,Unnamed: 1,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NEW_AMT_CREDIT_TO_AMT_INCOME,NEW_AMT_CREDIT_TO_AMT_ANNUITY,NEW_AMT_CREDIT_TO_AMT_GOODS_PRICE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_CONTRACT_TYPE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_HOUSING_TYPE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_ORGANIZATION_TYPE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_EDUCATION_TYPE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_GENDER,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_FAMILY_STATUS,NEW_AMT_CREDIT_TO_MEAN_AMT_INCOME_BY_AGE_GROUP,NEW_AMT_INCOME_BY_AGE_GROUP,NEW_AMT_INCOME_BY_CNT_CHILD,NEW_AMT_INCOME_BY_CNT_FAM_MEMBERS,NEW_AMT_INCOME_BY_AGE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_CONTRACT_TYPE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_HOUSING_TYPE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_ORGANIZATION_TYPE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_EDUCATION_TYPE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_GENDER,NEW_AMT_INCOME_TO_MEAN_AMT_INCOME_BY_AGE_GROUP,NEW_DOC_FLAG_MEAN,NEW_DOC_FLAG_SUM,NEW_CONTACT_FLAG_MEAN,NEW_CONTACT_FLAG_SUM,NEW_ADDRESS_FLAG_MEAN,NEW_ADDRESS_FLAG_SUM,NEW_OWN_CAR_REALTY_COMBINATION,NEW_AGE_TO_MEAN_AGE_BY_FLAG_OWN_REALTY,NEW_AGE_TO_MEAN_AGE_BY_FLAG_OWN_CAR,NEW_AGE_TO_MEAN_AGE_BY_HOUSING_TYPE,NEW_DAYS_EMPLOYED_TO_DAYS_BIRTH,NEW_DAYS_REGISTRATION_TO_DAYS_BIRTH,NEW_OWN_CAR_AGE_TO_DAYS_BIRTH,NEW_OWN_CAR_AGE_TO_DAYS_EMPLOYED,NEW_DAYS_LAST_PHONE_CHANGE_TO_DAYS_BIRTH,NEW_DAYS_LAST_PHONE_CHANGE_TO_DAYS_EMPLOYED,NEW_CNT_CHILD_TO_CNT_FAM_MEMBERS,NEW_EXT_SOURCES_MEAN,NEW_EXT_SOURCES_STD,NEW_DAYS_CHANGE_MEAN,NEW_REGION_RATING_CLIENT_MEAN,NEW_30_CNT_SOCIAL_CIRCLE_MEAN,NEW_60_CNT_SOCIAL_CIRCLE_MEAN,bureau_DAYS_CREDIT_sum,bureau_DAYS_ENDDATE_FACT_sum,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_ENDDATE_FACT_min,bureau_DAYS_ENDDATE_FACT_mean,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_UPDATE_sum,bureau_DAYS_ENDDATE_FACT_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_UPDATE_max,bureau_CNT_CREDIT_PROLONG_min,bureau_CREDIT_DAY_OVERDUE_min,bureau_AMT_CREDIT_SUM_OVERDUE_min,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_sum,bureau

I chose 90% colinearity as a threshold, you can try other values. Spoiler, lots of features are going to be dropped.

In [8]:
to_drop=[column for column in upper.columns if any(upper[column] > 0.9)]
print(f'Correlation: {len(to_drop)} will be removed')

Correlation: 620 will be removed


In [9]:
train=train.drop(to_drop, axis=1)
train.shape

(307511, 749)

In [8]:
# gc.enable()
# del corr_mat, upper
# gc.collect()

In the very first step of FS, the dataset lost approx 40% of features. How will it affect modelling results?

In [16]:
# train=train.iloc[:, 1:]

In [17]:
def quick_cv(df):
    X, y=df.drop(['TARGET'], axis=1), df['TARGET']
    cat_cols, num_cols=X.select_dtypes(include=['category', 'object']).columns, X.select_dtypes('number').columns

    ohe=OneHotEncoder(sparse=False, handle_unknown='ignore')
    col_tr=ColumnTransformer([
        ('cat', ohe, cat_cols),
        ('num', 'passthrough', num_cols)
    ])

    res, importances=custom_lgbm_cv(X, y, col_tr)
    return res, importances


In [18]:
res, importances=quick_cv(train)
res

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[410]	train's auc: 0.871944	test's auc: 0.785543
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[386]	train's auc: 0.868292	test's auc: 0.789011
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[429]	train's auc: 0.875703	test's auc: 0.7827
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[322]	train's auc: 0.857741	test's auc: 0.788504
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[442]	train's auc: 0.876365	test's auc: 0.787083


,Train AUC,Test AUC
0,0.8719,0.7855
1,0.8683,0.7890
2,0.8757,0.7827
3,0.8577,0.7885
4,0.8764,0.7871
Avg,0.8700,0.7866


Well, there are much less features, training is faster and results are almost the same as with the full data. Great! 

## Missing features

Now I'll remove mostly empty features, i.e., features with 70-90% of NaNs. I'll re-use `miss_table` function from EDA notebook.

In [19]:
mt=miss_table(train)
mt.head()

There are 683/748 columns with missing values
Distribution by dtypes:
float32    683
Name: Dtype, dtype: int64


,Count,Percent,Dtype
previous_RATE_INTEREST_PRIVILEGED_min,302902.0000,98.5012,float32
loan_card_AMT_PAYMENT_CURRENT_min_min,246451.0000,80.1438,float32
loan_card_AMT_DRAWINGS_ATM_CURRENT_max_min,246371.0000,80.1178,float32
loan_card_CNT_DRAWINGS_ATM_CURRENT_mean_min,246371.0000,80.1178,float32
loan_card_CNT_DRAWINGS_ATM_CURRENT_min_min,246371.0000,80.1178,float32


In [20]:
thresh=80
to_drop=mt.loc[mt['Percent']>thresh].index
print(f'{len(to_drop)} features with {thresh}% of NaNs will be removed')

16 features with 80% of NaNs will be removed


A few features got dropped, let's see modelling results.

In [21]:
train=train.drop(to_drop, axis=1)
train.shape

(307511, 732)

In [22]:
res, importances=quick_cv(train)
res

Training until validation scores don't improve for 100 rounds


KeyboardInterrupt: 

Performance remains stable (roughly the same), even though we got less and les featues. Perfect!

## Low Variance

Features with zero or very low (here threshold is subjective) variance can be dropped too. Let's try it out.

In [25]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()
numeric_data=train.iloc[:, 1:].select_dtypes('number').reset_index(drop=True)
numeric_data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [27]:
for i in numeric_data.columns:
    numeric_data[i].fillna(value=numeric_data[i].mean(), inplace=True) #replace NaN with mean of dimension
    numeric_data[i] = scaler.fit_transform(numeric_data[i].values.reshape(-1,1)) 

train_vars=numeric_data.var().sort_values()

In [30]:
to_drop=train_vars[train_vars<0.00005].index
to_drop.shape

(112,)

In [32]:
train=train.drop(to_drop, axis=1)
train.shape

In [18]:
res, importances=quick_cv(train)
res

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[270]	train's auc: 0.838817	test's auc: 0.777574
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[291]	train's auc: 0.842919	test's auc: 0.778338
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[283]	train's auc: 0.842254	test's auc: 0.772549
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[262]	train's auc: 0.837949	test's auc: 0.77896
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[426]	train's auc: 0.863819	test's auc: 0.780061


,Train AUC,Test AUC
0,0.8388,0.7776
1,0.8429,0.7783
2,0.8423,0.7725
3,0.8379,0.7790
4,0.8638,0.7801
Avg,0.8452,0.7775


In [20]:
# train.to_csv('../data/train_filtered.csv', chunksize=500)

In [21]:
# https://www.kaggle.com/code/ogrellier/feature-selection-with-null-importances/notebook
# https://www.kdnuggets.com/2019/10/feature-selection-beyond-feature-importance.html
# https://datascience.stackexchange.com/questions/12554/does-xgboost-handle-multicollinearity-by-itself
# null importance??

## Feature importance

The last thing I'll try out is selecting most important features according to LightGBM. Yes, feature importances can be used for FS too.

First, I'll define a function for getting feature importances in 2 ways: `gain` and `split` (default is this one). You can [refer to LightGBM docs](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.Booster.html#lightgbm.Booster.feature_importance) for more information. Note, that I won't OHE features, but will tell LightGBM which ones are categorical. LightGBM [can handle](https://lightgbm.readthedocs.io/en/latest/Features.html#optimal-split-for-categorical-features) categorical features natively.

In [19]:
def feature_imp_cv(features, target, k=5):
    metric_df=pd.DataFrame(columns=['Train AUC', 'Test AUC'])
    feat_importances_gain,feat_importances_split=[], []
    kfold=StratifiedKFold(k)
    
    for f, (tr, te) in enumerate(kfold.split(features, y=target)):
        X_train, y_train=features.iloc[tr, :], target.iloc[tr]
        X_test, y_test=features.iloc[te, :], target.iloc[te]

        weight=np.count_nonzero(y_train==0)/np.count_nonzero(y_train==1)

        params={'num_boost_round': 10000,
                'objective': 'binary',
                'scale_pos_weight': weight,
                'metric': 'auc',
                'learning_rate': 0.05,
                'reg_alpha': 0.1,
                'reg_lambda': 0.1,
                'subsample': 0.8,
                'n_jobs': -1,
                'random_state': 5,
                'verbose': -1}

        dtrain=lgb.Dataset(X_train, label=y_train)
        dval=lgb.Dataset(X_test, label=y_test)

        model=lgb.train(
                params=params,
                train_set=dtrain,
                valid_sets=[dtrain, dval],
                valid_names=['train', 'test'],
                categorical_feature=list(features.select_dtypes('category').columns),
                callbacks=[lgb.early_stopping(100, verbose=-1), lgb.log_evaluation(-1)])
    
        test_score, train_score=model.best_score['test']['auc'], model.best_score['train']['auc']
        metric_df.loc[f]=[train_score, test_score]
        
        feat_importances_gain.append(model.feature_importance(importance_type='gain'))
        feat_importances_split.append(model.feature_importance(importance_type='split'))

    
    feat_importances_gain=np.array(feat_importances_gain).mean(axis=0)
    feat_importances_split=np.array(feat_importances_split).mean(axis=0)
    feat_importances_df=pd.DataFrame({'feature': list(features.columns),
                                        'importance (gain)': feat_importances_gain,
                                        'importance (split)': feat_importances_split,})
    metric_df.loc['Avg']=[metric_df['Train AUC'].mean(), metric_df['Test AUC'].mean()]
    return metric_df, feat_importances_df

In [20]:
import re
train=train.rename(columns=lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [21]:
X, y=train.drop(['TARGET', 'SK_ID_CURR'], axis=1), train['TARGET']

res, imps=feature_imp_cv(X, y)
res

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[202]	train's auc: 0.833789	test's auc: 0.775633
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[303]	train's auc: 0.85302	test's auc: 0.777657
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[208]	train's auc: 0.83577	test's auc: 0.771201
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[352]	train's auc: 0.861746	test's auc: 0.777664
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[412]	train's auc: 0.870351	test's auc: 0.780385


,Train AUC,Test AUC
0,0.8338,0.7756
1,0.8530,0.7777
2,0.8358,0.7712
3,0.8617,0.7777
4,0.8704,0.7804
Avg,0.8509,0.7765


In [22]:
imps=imps.sort_values(by='importance (gain)', ascending=False).reset_index(drop=True)
imps

,feature,importance (gain),importance (split)
0,EXT_SOURCE_3,324262.4357,262.2000
1,EXT_SOURCE_2,260893.6252,247.4000
2,ORGANIZATION_TYPE,184568.2692,1401.0000
3,EXT_SOURCE_1,96844.1151,301.4000
4,client_installments_AMT_PAYMENT_min_sum,63888.0072,194.0000
...,...,...,...
498,previous_CODE_REJECT_REASON_SCOFR_sum,0.0000,0.0000
499,previous_FLAG_LAST_APPL_PER_CONTRACT_N_sum,0.0000,0.0000
500,previous_NAME_GOODS_CATEGORY_AutoAccessories_sum,0.0000,0.0000
501,previous_NAME_CASH_LOAN_PURPOSE_Urgentneeds_sum,0.0000,0.0000


### Zero Importance Features

Features with zero importance can be dropped without much hesitation. Let's test it out.

In [23]:
feat_zero_imp=imps[(imps['importance (split)']==0) & (imps['importance (gain)']==0)]['feature'].values
feat_zero_imp.shape

(66,)

In [24]:
train=train.drop(feat_zero_imp, axis=1)
train.shape

(307511, 439)

In [ ]:
res, importances=quick_cv(train)
res

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[256]	train's auc: 0.83625	test's auc: 0.778258
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[294]	train's auc: 0.843476	test's auc: 0.778333
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[350]	train's auc: 0.853485	test's auc: 0.772599
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[293]	train's auc: 0.84315	test's auc: 0.779272
Training until validation scores don't improve for 100 rounds


### Low Importance Features

This part is a bit tricky, as you need to pick the treshold. I've picked very high one of 0.999. Hence, I expect very little information loss. But with lower thresholds, the risk of data loss (and folliwing performance loss) rises.

In [ ]:
aa=np.cumsum(imps.iloc[:, 1:])/np.sum(imps.iloc[:, 1:])
aa

In [124]:
drop_index=np.argwhere(aa['importance (gain)'].values>0.999)[0][0]
drop_index

376

In [125]:
low_zero_imp=imps.iloc[drop_index:, :]['feature'].values
low_zero_imp

array(['client_credit_AMT_INST_MIN_REGULARITY_min_min',
       'previous_NAME_GOODS_CATEGORY_Medicine_mean',
       'client_cash_NAME_CONTRACT_STATUS_Approved_mean_max',
       'previous_NAME_CLIENT_TYPE_XNA_mean',
       'previous_NAME_SELLER_INDUSTRY_Construction_sum',
       'previous_PRODUCT_COMBINATION_CardStreet_sum',
       'previous_NAME_SELLER_INDUSTRY_Jewelry_mean', 'FLAG_OWN_REALTY',
       'previous_NAME_GOODS_CATEGORY_Mobile_sum',
       'AMT_REQ_CREDIT_BUREAU_HOUR',
       'previous_NAME_SELLER_INDUSTRY_Clothing_sum',
       'previous_NAME_CASH_LOAN_PURPOSE_Buyingahome_mean',
       'previous_NAME_GOODS_CATEGORY_Furniture_sum',
       'client_credit_AMT_DRAWINGS_OTHER_CURRENT_sum_sum',
       'previous_FLAG_LAST_APPL_PER_CONTRACT_N_mean', 'FLAG_DOCUMENT_6',
       'previous_NAME_CASH_LOAN_PURPOSE_Education_mean',
       'previous_RATE_INTEREST_PRIVILEGED_sum',
       'previous_PRODUCT_COMBINATION_CashStreetlow_sum',
       'previous_NAME_SELLER_INDUSTRY_Furniture_sum', 'F

In [126]:
train=train.drop(low_zero_imp, axis=1)
train.shape

(307511, 378)

In [127]:
res, importances=quick_cv(train)
res

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[253]	train's auc: 0.835615	test's auc: 0.777766
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[258]	train's auc: 0.837459	test's auc: 0.778217
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[281]	train's auc: 0.841689	test's auc: 0.773277
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[304]	train's auc: 0.84468	test's auc: 0.778509
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[428]	train's auc: 0.864528	test's auc: 0.779141


,Train AUC,Test AUC
0,0.8356,0.7778
1,0.8375,0.7782
2,0.8417,0.7733
3,0.8447,0.7785
4,0.8645,0.7791
Avg,0.8448,0.7774
